In [3]:
import pandas as pd

In [71]:
df = pd.read_csv(r'R:\4K\2014\Trip Generation\Trip Rates\trip_2014_survey_gps.csv')
parcel = pd.read_csv(r'L:\LODES\soundcast_2014_lodes\inputs\buffered_parcels.txt', sep=' ')

#export_trip_hh = pd.merge(export_trip_hh, parcel, left_on='opcl', right_on='parcelid')
#export_trip_hh = pd.merge(export_trip_hh, parcel, left_on='dpcl', right_on='parcelid', suffixes=['_o','_d'])

In [72]:
# Sum employment variables by TAZ

# Take a weighted average by emp, pop, and emp+pop for other variables


In [73]:
# Calculate TAZ totals for households, employment
totals_by_taz = parcel.groupby('taz_p').sum()[[
       'hh_p', u'stugrd_p', u'stuhgh_p', u'stuuni_p', u'empedu_p',
       u'empfoo_p', u'empgov_p', u'empind_p', u'empmed_p', u'empofc_p',
       u'empret_p', u'empsvc_p', u'empoth_p', u'emptot_p']]
totals_by_taz.columns = [i.split('_p')[0]+'_taz_tot' for i in totals_by_taz.columns]
totals_by_taz = totals_by_taz.reset_index()


In [74]:
# Calculate a weighted average for each TAZ
# based on the parcel buffers
# This avoids bias based on TAZ boundaries

# Weighted by parcel population, parcel employment, and combination of both
# For 1st buffer (0.25 miles?)

wt_fields = ['stugrd_',
       u'stuhgh_', u'stuuni_', u'empedu_', u'empfoo_', u'empgov_',
       u'empind_', u'empmed_', u'empofc_', u'empret_', u'empsvc_',
       u'empoth_', u'emptot_', u'parkdy_', u'parkhr_', u'ppricdy',
       u'pprichr']

for field in wt_fields:
    
    # Get numerator of weights (values within buffered areas)
    # to be divided by denominator of total hh/emp/hh+emp
    
    # Weight by parcel population, for 0.25 and 0.5 mile buffers
    parcel[field+'hh_wt1_tot'] = parcel[field+'1']*parcel['hh_p']    # .25 mile
    parcel[field+'hh_wt2_tot'] = parcel[field+'2']*parcel['hh_p']    # .5 mile
    
    # Weight by total parcel employment
    parcel[field+'emp_wt1_tot'] = parcel[field+'1']*parcel['emptot_p']    # .25 mile
    parcel[field+'emp_wt2_tot'] = parcel[field+'2']*parcel['emptot_p']    # .5 mile
    
    # Weight by households + employment
    parcel[field+'hhemp_wt1_tot'] = parcel[field+'1']*(parcel['emptot_p']+parcel['hh_p'])# .25 mile
    parcel[field+'hhemp_wt2_tot'] = parcel[field+'2']*(parcel['emptot_p']+parcel['hh_p'])# .5 mile

In [75]:
# Compute weighted average for TAZ based on 

parcel['emptot_plus_hh'] = parcel['emptot_p']+parcel['hh_p']

for field in wt_fields:
    print field
    # Weights based on household size
    totals_by_taz[field+'hh_wt1_avg'] = parcel.groupby('taz_p').sum()[field+'hh_wt1_tot']/parcel.groupby('taz_p').sum()['hh_p']
    totals_by_taz[field+'hh_wt2_avg'] = parcel.groupby('taz_p').sum()[field+'hh_wt2_tot']/parcel.groupby('taz_p').sum()['hh_p']
    
    # Weights based on employment
    totals_by_taz[field+'emp_wt1_avg'] = parcel.groupby('taz_p').sum()[field+'emp_wt1_tot']/parcel.groupby('taz_p').sum()['hh_p']
    totals_by_taz[field+'emp_wt2_avg'] = parcel.groupby('taz_p').sum()[field+'emp_wt2_tot']/parcel.groupby('taz_p').sum()['hh_p']
    
    # weights based on housheholds + employment
    totals_by_taz[field+'hhemp_wt1_avg'] = parcel.groupby('taz_p').sum()[field+'hhemp_wt1_tot']/parcel.groupby('taz_p').sum()['emptot_plus_hh']
    totals_by_taz[field+'hhemp_wt2_avg'] = parcel.groupby('taz_p').sum()[field+'hhemp_wt2_tot']/parcel.groupby('taz_p').sum()['emptot_plus_hh']

stugrd_
stuhgh_
stuuni_
empedu_
empfoo_
empgov_
empind_
empmed_
empofc_
empret_
empsvc_
empoth_
emptot_
parkdy_
parkhr_
ppricdy
pprichr


In [93]:
# Join totals by taz to the trip file
trip = pd.read_csv(r'R:\4K\2014\Trip Generation\Trip Rates\trip_2014_survey_gps.csv')

df = pd.merge(trip, totals_by_taz, left_on='otaz', right_on='taz_p', how='left')
df = pd.merge(df, totals_by_taz, left_on='dtaz', right_on='taz_p', how='left', suffixes=['_o','_d'])
df.to_csv(r'R:\4K\2014\Trip Generation\Trip Rates\trip_2014_survey_gps_lu_vars.csv', index=False)